In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [2]:
# Load dataset
data_dir = 'fruits'
image_size = (224, 224)  # ResNet50 expects 224x224 images
batch_size = 32

In [3]:
train_ds = keras.utils.image_dataset_from_directory(
    directory=data_dir,
    labels="inferred",
    label_mode="int",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=123
)
val_ds = keras.utils.image_dataset_from_directory(
    directory=data_dir,
    labels="inferred",
    label_mode="int",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    seed=123
)

Found 16000 files belonging to 16 classes.
Using 12800 files for training.
Found 16000 files belonging to 16 classes.
Using 3200 files for validation.


In [4]:
class_names = train_ds.class_names

# Apply ResNet-specific preprocessing
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))

# Prefetch for efficiency
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [5]:
# Build the transfer learning model
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze base model for fast training

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 49s 1us/step


In [6]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

In [7]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [8]:
# Train the model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3  # 3-5 is typically enough for good results with transfer learning
)

Epoch 1/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 641s 2s/step - accuracy: 0.9270 - loss: 0.2471 - val_accuracy: 0.9981 - val_loss: 0.0157
Epoch 2/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 632s 2s/step - accuracy: 0.9895 - loss: 0.0346 - val_accuracy: 0.9950 - val_loss: 0.0168
Epoch 3/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 688s 2s/step - accuracy: 0.9951 - loss: 0.0186 - val_accuracy: 0.9984 - val_loss: 0.0058


In [ ]:
# Save the trained model to disk
model.save('fruit_classifier_resnet50.keras')  